In [ ]:
# !pip install ta
# !pip install python-binance
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from binance.client import Client
import pandas as pd
import math
import ta
from ta import add_all_ta_features


# # Configuración de la API de Binance
# api_key = 'CNrMcRI2PaYT46mJDVI0bBz1Pfpe8tne8jUpm1DYk8bkFUNfEGhVlVNOYT2nBWQQ'
# api_secret = 'pqnzlpng07bLJn2OyqhXOikUyc98e9qo1q54yhQeDkEnBx4Hx4ykwmKOeaOWeWtS'
# client = Client(api_key, api_secret)

In [ ]:
def get_historic_data(symbol, interval, start, end):
    """Obtiene datos históricos para un símbolo en un intervalo y lookback especificado."""
    data = client.get_historical_klines(symbol, interval, start, end)

    # Convertir la lista de datos en DataFrame y preparar columnas
    df = pd.DataFrame(data).iloc[:, :6]
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    price_columns = ['open', 'high', 'low', 'close', 'volume']
    df[price_columns] = df[price_columns].apply(pd.to_numeric, errors='coerce')
    df.columns = [col + f"_{interval}" if col != 'timestamp' else col for col in df.columns]
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

# Esta función reemplaza get_historic_5_min y get_historic_1_hour
# Ejemplo: df_5m = get_historic_data('BTCUSDT', '5m', "15 hours ago UTC")


In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'
df_5min = get_historic_data('BTCUSDT', '5m', start_date, end_date)
df_1hour = get_historic_data('BTCUSDT', '1h', start_date, end_date)
result_df = rsi_atr(df_5min, df_1hour, 14)

In [ ]:
df = add_all_ta_features(df_1hour, open="open_1h", high="high_1h", low="low_1h", close="close_1h", volume="volume_1h", fillna=True)

In [ ]:
import pandas as pd
df = pd.read_csv("BTC_2022_5m.csv", index_col=0)

In [ ]:
df

,open_5m,high_5m,low_5m,close_5m,volume_5m,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,46216.93,46391.49,46208.37,46321.34,185.67558,43.417586,185.675580,0.233836,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,46321.340000,0.000000,0.000000,0.000000
2022-01-01 00:05:00,46321.34,46527.26,46280.00,46371.11,123.43577,10.948677,309.111350,0.035420,6143.398273,2.077264e+10,...,0.008570,0.001714,0.006856,-2.742105,-0.548421,-2.193684,46343.479173,0.107445,0.107387,0.107445
2022-01-01 00:10:00,46369.79,46394.00,46276.22,46332.51,77.54574,7.525024,231.565610,0.019462,4838.160582,-1.040713e+10,...,0.008540,0.003079,0.005461,-7.185309,-1.875799,-5.309510,46338.624548,-0.083241,-0.083276,0.024114
2022-01-01 00:15:00,46332.52,46332.52,46236.27,46293.90,101.14315,27.501453,130.422460,0.056379,3589.118067,-4.826149e+09,...,0.001771,0.002818,-0.001047,-9.703462,-3.441331,-6.262131,46318.933139,-0.083332,-0.083367,-0.059238
2022-01-01 00:20:00,46295.42,46421.27,46286.25,46395.53,135.32479,111.230033,265.747250,0.178504,5041.109544,6.920877e+09,...,0.013947,0.005044,0.008903,-9.906756,-4.734416,-5.172340,46352.642040,0.219532,0.219292,0.160164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 23:40:00,16612.26,16616.46,16606.49,16615.56,357.58357,-134994.867834,-81156.764508,0.154393,1698.416554,8.280833e+06,...,0.042154,0.031153,0.011001,-5.077036,-7.860096,2.783061,16597.242563,0.023176,0.023174,-64.129794
2022-12-30 23:45:00,16615.97,16618.75,16602.84,16604.58,431.22843,-135331.773515,-81587.992938,0.134617,779.373023,-2.508833e+06,...,0.041869,0.033296,0.008573,-4.330826,-7.154242,2.823416,16597.829304,-0.066083,-0.066104,-64.153498
2022-12-30 23:50:00,16604.58,16607.15,16600.96,16603.79,263.49543,-135354.334514,-81851.488368,0.121235,638.296678,-1.583352e+07,...,0.040790,0.034795,0.005995,-6.988977,-7.121189,0.132212,16598.119877,-0.004758,-0.004758,-64.155204


In [ ]:
import numpy as np
import pandas as pd
import random
import re

# Definición de los operadores, comparadores e indicadores
operators = [' + ', ' - ', ' * ', ' / ']
comparators = [' >= ', ' <= ', ' > ', ' < ']
# Asumiremos que df es tu DataFrame y que este contiene las columnas indicadoras
# Estos son ejemplos, debes reemplazarlos por los nombres reales de tus columnas
indicadores = list(df.columns)

# Genera un elemento aleatorio para el cromosoma
def generate_random_element():
    chromosome = []

    for ciclo in range(2):
        random.seed(random.randint(1,1000000000))
        element_type = random.choice(['operator', 'comparator'])

        if element_type in ['comparator']:
            left = "df.iloc[{i}]['%s']" % random.choice(indicadores)
            right = random.choice(["df.iloc[{i}]['%s']" % random.choice(indicadores), round(random.uniform(0.01, 100), 2)])
            comparation = random.choice(comparators)
            chromosome.append( f"{left} {comparation} {right}")

        elif element_type == 'operator':
            left = "df.iloc[{i}]['%s']" % random.choice(indicadores)
            right = random.choice(["df.iloc[{i}]['%s']" % random.choice(indicadores), round(random.uniform(0.01, 100), 2)])
            right_2 = random.choice(["df.iloc[{i}]['%s']" % random.choice(indicadores), round(random.uniform(0.01, 100), 2)])
            operation = random.choice(operators)
            comparation = random.choice(comparators)
            chromosome.append( f"{left} {operation} {right} {comparation} {right_2}")

    return chromosome

# Inicializa la población con cromosomas de longitud 1
def initialize_population(population_size):
    population = []

    while len(population) < population_size:
        chromosome = generate_random_element()  # i=0 para el inicio
        if chromosome not in population:
            population.append(chromosome)
        else:
            continue

    return population

def evaluate_chromosome(chromosome, df):
    balance = 1000  # Balance inicial
    commission = 0.00  # Comisión por operación como porcentaje
    position_open = False  # Indica si hay una posición abierta

    for j in range(1, len(df)):

        # Alternar entre condiciones de compra y venta según el estado de la posición
        if not position_open:
            # Construir y evaluar la expresión de compra
            buy_expression = chromosome[0].format(i=j)

            if eval(buy_expression):
#                 print(balance)
                balance = balance * (1-commission) / df.iloc[j]["close_5m"]
                position_open = True
        else:
            # Construir y evaluar la expresión de venta
            sell_expression = chromosome[1].format(i=j)
            if eval(sell_expression):
                balance = balance * (1-commission) * df.iloc[j]["close_5m"]
#                 print (balance)
                position_open = False  # Cerrar posición

    if balance == 1000:
        pass
    else:
        if balance > 0 and balance < 0.1:
            balance = balance * (1-commission) * df.iloc[j]["close_5m"]
        else:
            pass

#     print(balance)
#     print(chromosome[0], "\n")
#     print(chromosome[1], "\n")
    return balance

def select_parents(population, fitness):

    sorted_population = [x for _, x in sorted(zip(fitness, population), key=lambda pair: pair[0], reverse=True)]
    # Crear un diccionario donde cada clave es la concatenación de los elementos de la población con '//',
    pop_fitness_dict = {"------------- // -------------".join(map(str, indiv)): fit for indiv, fit in zip(population, fitness)}

    # Ordenar el diccionario por los valores de fitness, de mayor a menor
    sorted_pop_fitness_dict = dict(sorted(pop_fitness_dict.items(), key=lambda item: item[1], reverse=True))

    return sorted_population, sorted_pop_fitness_dict

def actualizar_diccionario_generaciones(diccionario_generaciones, poblacion_actualizada, fitness_actualizado):
    # Obtener el diccionario de la población actualizada con su fitness
    diccionario_actualizado = select_parents(poblacion_actualizada, fitness_actualizado)[1]

    # Actualizar el diccionario externo con los nuevos valores
    for clave, valor in diccionario_actualizado.items():
        diccionario_generaciones[clave] = valor

    # Ordenar el diccionario por los valores de fitness, de mayor a menor
    diccionario_generaciones = dict(sorted(diccionario_generaciones.items(), key=lambda item: item[1], reverse=True))

def build_child(comparators_parent, operators_parent, all_comparators):
    child = ""
    for i in range(len(comparators_parent)):
        comparator = random.choice(all_comparators)
        child += comparator
        if i < len(operators_parent):
            child += ' ' + operators_parent[i] + ' '
        all_comparators.remove(comparator)
    return child

def crossover(parent1, parent2):
    parts1 = re.split(r' (AND|OR) ', parent1)
    comparators1, operators1 = parts1[::2], parts1[1::2]

    parts2 = re.split(r' (AND|OR) ', parent2)
    comparators2, operators2 = parts2[::2], parts2[1::2]

    all_comparators = comparators1 + comparators2
    random.shuffle(all_comparators)

    child1 = build_child(comparators1, operators1, all_comparators.copy())
    child2 = build_child(comparators2, operators2, all_comparators)

    return child1, child2

# Mutación; ahora incluirá la lógica para agregar un "gen" en cada generación
def mutate(offspring, mutation_rate_pop):
    # Obtener índices de los elementos a mutar
    mutated_pop_indices = np.random.randint(0, len(offspring), size=mutation_rate_pop)

    for j in mutated_pop_indices:

        for index, parent in enumerate(offspring[j]):
            chromosome_parts = re.split(r'\s+and\s+|\s+or\s+', parent)

            if len(chromosome_parts)+1 < 20:
                new_gene_element = random.choice(generate_random_element()) # Solo se escoge un gen de forma azarosa
                new_gen = parent + random.choice([' and ', ' or ']) + new_gene_element
                offspring[j][index] = new_gen

    return offspring


# Algoritmo genético ajustado
def genetic_algorithm(df, population_size, generations, mutation_rate, elitismo):
    population = initialize_population(population_size)
    max_fitness = 1000
    diccionario_generaciones = {}
    for generation in range(generations):
        fitness = []
        for chromosome in population:
            result = evaluate_chromosome(chromosome, df)

            if result > max_fitness:
                max_fitness = result
                print(f"Nuevo mayor fitness encontrado: {result}")
                print(chromosome, "\n")
            fitness.append(result)

        selected_population = select_parents(population, fitness)[0]
        actualizar_diccionario_generaciones(diccionario_generaciones, population, fitness)

        offspring = selected_population[:int((elitismo*len(population))/100)] # Elitismo: Pasan el 5% de los mejores de la anterior población
        for parents in selected_population[:int((100-elitismo*len(population))/100)]: #Cruzan 95% primero de la anterior población
            childs = crossover(parents[0], parents[1])
            offspring.append([childs[0], childs[1]])

        mutation_rate_pop = int(population_size*mutation_rate)
        offspring = mutate(offspring, mutation_rate_pop)
        population = offspring

    return diccionario_generaciones


In [ ]:
# Ejemplo de uso
if __name__ == "__main__":

    # Parámetros del algoritmo genético
    elitismo = 10 # 5% de gente que pasa a la siguiente generación tal y como está MIN 2
    population_size = 10000 # MIN 10
    generations = 100  # Cambia esto según cuántas generaciones quieras correr
    mutation_rate = 0.3 # % de la población que muta de forma azarosa

    # Ejecución del algoritmo genético
    best_chromosome = genetic_algorithm(df, population_size, generations, mutation_rate, elitismo)

    print("\n", f"Mejor expresión encontrada: {next(iter(best_chromosome.items()))}")


Nuevo mayor fitness encontrado: 1074.6062740722289
["df.iloc[{i}]['momentum_kama']  -  19.18  >  40.31", "df.iloc[{i}]['trend_sma_slow']  <  df.iloc[{i}]['volume_em']"] 

Nuevo mayor fitness encontrado: 1239.925324031589
["df.iloc[{i}]['trend_aroon_ind']  *  71.99  <  df.iloc[{i}]['volatility_bbw']", "df.iloc[{i}]['trend_ema_slow']  >  df.iloc[{i}]['volatility_kchi']"] 



<string>:1: RuntimeWarning: divide by zero encountered in scalar divide
<string>:1: RuntimeWarning: invalid value encountered in scalar divide


Nuevo mayor fitness encontrado: 1913.4487827966007
["df.iloc[{i}]['volume_vwap']  >=  df.iloc[{i}]['high_5m']", "df.iloc[{i}]['volume_em']  /  37.15  >  df.iloc[{i}]['trend_adx']"] 



In [ ]:
Nuevo mayor fitness encontrado: 1048.6019817114934
["df.iloc[{i}]['momentum_rsi']  <  71.66", "df.iloc[{i}]['momentum_stoch_signal']  -  61.19  <=  87.75"]

Nuevo mayor fitness encontrado: 1092.0043716109149
["df.iloc[{i}]['trend_cci']  /  96.88  >  df.iloc[{i}]['volume_fi']", "df.iloc[{i}]['volatility_bbl']  <=  df.iloc[{i}]['trend_ema_fast']"]

<string>:1: RuntimeWarning: divide by zero encountered in scalar divide
Nuevo mayor fitness encontrado: 1188.9512424092218
["df.iloc[{i}]['volume_sma_em']  +  df.iloc[{i}]['momentum_kama']  <=  68.22", "df.iloc[{i}]['trend_adx_pos']  <  89.48"]

Nuevo mayor fitness encontrado: 1244.009943114343
["df.iloc[{i}]['volatility_kcl']  -  df.iloc[{i}]['volatility_bbm']  >=  26.33", "df.iloc[{i}]['high_5m']  *  df.iloc[{i}]['volume_em']  <  18.97"]

<string>:1: RuntimeWarning: invalid value encountered in scalar divide
Nuevo mayor fitness encontrado: 1438.0932457068216
["df.iloc[{i}]['trend_aroon_up']  <=  18.52", "df.iloc[{i}]['trend_sma_slow']  <  df.iloc[{i}]['volatility_bbm']"]

Nuevo mayor fitness encontrado: 1801.273416925007
["df.iloc[{i}]['volume_em']  <  df.iloc[{i}]['others_dr']", "df.iloc[{i}]['volatility_dch']  -  69.68  >=  47.06"]

Nuevo mayor fitness encontrado: 1913.4487827966007
["df.iloc[{i}]['volume_vwap']  >=  df.iloc[{i}]['high_5m']", "df.iloc[{i}]['volume_em']  /  37.15  >  df.iloc[{i}]['trend_adx']"]
